In [ ]:
import chess
import chess.engine

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [ ]:
from generate_data import *

In [ ]:
class ChessDataset(Dataset):
    """Chess Positions Evaluation Dataset"""

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        position = np.array(self.df.iloc[idx, :-1])
        eval_ = np.array(self.df.iloc[idx, -1]/100)
        
        sample = {'position': torch.from_numpy(position), 'eval': torch.from_numpy(eval_)}

        return sample

In [ ]:
dataset = ChessDataset(csv_file='data/stockfish_depth0e.csv')

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.fc1 = nn.Linear(513, 64)
        # self.conv1 = nn.Conv1d(64, 8, 3) #input, output channels, kernel_size
        # self.conv2 = nn.Conv2d(8, 8, 3)
        # self.conv3 = nn.Conv2d(8, 8, 3)
        # self.conv4 = nn.Conv2d(8, 8, 3)
        self.fc2 = nn.Linear(64, 32) 
        self.fc3 = nn.Linear(32,8)
        self.fc4 = nn.Linear(8, 1) 

    def forward(self, x):
        x = self.fc1(x)
        # x = self.conv1(x)
        # x = self.conv2(x)
        # x = self.conv3(x)
        # x = self.conv4(x)   
        x = torch.reshape(x, (-1,)) # flattens into 1d tensor
        x = self.fc2(x)
        x = self.fc3(F.relu(x))
        x = self.fc4(torch.sigmoid(x))
        #x = self.fc1(x)
        #x = self.fc2(x)

        return x

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)

net = Net()#.cuda()

# Load model
net.load_state_dict(torch.load('chess_ai0.pt'))
net.eval()

In [ ]:
t = torch.tensor([[1., -1.], [1., -1.]])
t = t.cuda()
t.get_device()

# TRAINING DATA

In [ ]:
#torch.cuda.get_device_name(device)

In [ ]:
trainloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0) 
#num_workers = 0 doesn't use GPU. GPU seems to be a lot slower than CPU,.
#data set is prob too big or my laptop nvidia GPU is just trash

optimizer = optim.Adam(net.parameters(), lr = 0.0009) #lr = 0.001
criterion = nn.MSELoss() 

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['position'].reshape(1, 513).float()#.cuda()
        labels = data['eval'].float()#.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 400 == 399:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 400))
            running_loss = 0.0

print('Finished Training')

In [10]:
def compare_eval():
    board = generate_rand_board()
    #print(convert_board(board))
    torch_board = torch.from_numpy(convert_board(board).reshape(1, 513)).float()
    net_eval = net(torch_board)
    sf_eval = stockfish_evaluation(board)
    print(f"NN: {net_eval} Stockfish: {sf_eval/100}")
    

In [11]:
for _ in range(5):
    compare_eval()

NN: tensor([0.8682], grad_fn=<AddBackward0>) Stockfish: 2.09
NN: tensor([2.4021], grad_fn=<AddBackward0>) Stockfish: 0.99
NN: tensor([-0.5156], grad_fn=<AddBackward0>) Stockfish: 5.47
NN: tensor([6.3615], grad_fn=<AddBackward0>) Stockfish: 18.01
NN: tensor([7.0650], grad_fn=<AddBackward0>) Stockfish: 10.78


In [12]:
# Choosing Best Move with Minimax

In [13]:
def nn_prediction(position: chess.Board):
    board_rep = convert_board(position)
    torch_board = torch.from_numpy(board_rep.reshape(1, 513)).float()
    return net(torch_board)


In [14]:

        

def minimax(position: chess.Board, maximizing_player, depth: int = 5): # alpha: int = -1e9, beta: int = 1e9 
    """
    Minimax with alpha-beta pruning to save time
    """
    #just used minimax from connectx kaggle agent

    alpha = -1000 #could be -np.Inf
    beta = 1000
    if depth == 0 or position.is_game_over():
        return nn_prediction(position)

    if maximizing_player:
        value = -np.Inf
        for move in position.legal_moves:
            pos_copy = position.copy()
            if not position.remove_piece_at(move.from_square).color == chess.BLACK:
                position = pos_copy.copy()
                continue
            position.push(move)
            eval_tensor = minimax(position, False, depth - 1)
            if torch_eval > value:
                value = eval_tensor
            position.pop()
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value

    else:
        value = np.Inf
        for move in position.legal_moves:
            pos_copy = position.copy()
            if position.remove_piece_at(move.from_square).color == chess.WHITE:
                position = pos_copy.copy()
                continue
            position.push(move)
            value = min(value, minimax(position, True, depth - 1))
            position.pop()
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value

In [15]:
def find_best_move(position: chess.Board, depth: int = 0):
    best_evaluation = -1000
    best_move = None
    for move in position.legal_moves:
        if best_move == None:
            best_move = move
        
        #print(move)
        value = -1000

        # pos_copy = position.copy()
        # if position.remove_piece_at(move.from_square).color == chess.WHITE:
        #         position = pos_copy.copy()
        #         continue
        position.push(move)
        value = nn_prediction(position)
        if depth > 0:
            if position.turn == chess.WHITE:
                value = minimax(position, False, depth - 1) 
        print(move,value)
        position.pop()
        if value > best_evaluation:
            best_evaluation = value
            best_move = move
    return best_move


In [20]:
def play_game():
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")    
    while not board.is_game_over():   
        nn_move = find_best_move(board,24) #depth is working ?
        board.push(nn_move)
        print(board)
        print(nn_move)
        moved = False
        while not moved:
            try:
                sqr = input('your move')
                if sqr == 'q' or sqr == 'quit':
                    return
                if sqr[0] == 'n':
                    sqr = 'N' + sqr[1:]
                board.push_san(sqr)
                moved = True
            except:
                moved = False


In [22]:
play_game()

Backward0>)
e7h7 tensor([-2.1566], grad_fn=<AddBackward0>)
e7g7 tensor([0.4514], grad_fn=<AddBackward0>)
e7f7 tensor([1.1550], grad_fn=<AddBackward0>)
e7d7 tensor([-4.2814], grad_fn=<AddBackward0>)
e7c7 tensor([-3.8046], grad_fn=<AddBackward0>)
e7b7 tensor([-2.4133], grad_fn=<AddBackward0>)
e7a7 tensor([2.4528], grad_fn=<AddBackward0>)
e7e6 tensor([2.6326], grad_fn=<AddBackward0>)
e7e5 tensor([2.0135], grad_fn=<AddBackward0>)
e7e4 tensor([2.0401], grad_fn=<AddBackward0>)
e7e3 tensor([5.1969], grad_fn=<AddBackward0>)
e7e2 tensor([-0.5165], grad_fn=<AddBackward0>)
e7e1 tensor([-0.8018], grad_fn=<AddBackward0>)
d2e3 tensor([2.5440], grad_fn=<AddBackward0>)
d2d3 tensor([4.4889], grad_fn=<AddBackward0>)
d2c3 tensor([4.6439], grad_fn=<AddBackward0>)
d2e1 tensor([4.2172], grad_fn=<AddBackward0>)
d2d1 tensor([2.4185], grad_fn=<AddBackward0>)
d2c1 tensor([-0.2841], grad_fn=<AddBackward0>)
a1a8 tensor([2.0502], grad_fn=<AddBackward0>)
a1a7 tensor([2.4885], grad_fn=<AddBackward0>)
a1a6 tensor([1.

In [ ]:
# ended in a draw vs chess.com 700 elo bot. played like a 100 though     